In [37]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

import datetime

In [38]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [39]:
actions = np.array(["walking","boxing","handwaving"])

In [40]:
X = np.load("./X1.npy",allow_pickle = True)
y = np.load("./y1.npy", allow_pickle = True)
print(X)
#Chia dữ liệu
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

[[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.63365924  0.14614753 -0.14069085 ...  0.78772527  0.25177038
    0.8931123 ]
  [ 0.61432379  0.14053936 -0.14560398 ...  0.78763813  0.25283921
    0.89421827]
  [ 0.59513819  0.1403195  -0.18441123 ...  0.78761786  0.35164031
    0.8965258 ]]

 [[ 0.5766573   0.13791214 -0.17605722 ...  0.78489572  0.32537073
    0.88771462]
  [ 0.56357151  0.13555603 -0.09422498 ...  0.78301948  0.33317456
    0.89259863]
  [ 0.54559159  0.13458501 -0.07521461 ...  0.78399187  0.27583307
    0.88933361]
  ...
  [ 0.1227383   0.11517255 -0.14762533 ...  0.75696534  0.2958594
    0.95440036]
  [ 0.10444719  0.1131096  -0.12361038 ...  0.75356609  0.31085479
    0.94651288]
  [ 0.09093846  0.11017571 -0.12647308 ...  0.75333977  0.30464977
    0.93308228]]

 [[ 0.0726513

In [41]:
X.shape

(46, 30, 132)

In [42]:
X_train.shape

(36, 30, 132)

In [43]:
y_train.shape

(36, 3)

In [44]:
actions.shape[0]

3

In [45]:
import torch.optim as optim
import torch.nn as nn
from LSTM import LSTM_model
from torch.utils.data import Dataset, DataLoader
import torch

In [46]:
class MyDataset(Dataset):
    def __init__(self, data, labels = None):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        x = self.data[index]
        if self.labels is None:
            y = None
        else:
            y = self.labels[index]
        return x, y

    def __len__(self):
        return len(self.data)

In [47]:
# 创建数据集对象
dataset = MyDataset(X_train, y_train)
test = MyDataset(X_test, y_test)
# 创建数据加载器对象
train_loader = DataLoader(dataset, batch_size=4, shuffle=False, drop_last=True)
test_loader = DataLoader(test, batch_size=4, shuffle=False, drop_last=True)

In [48]:
# 定义模型和优化器
model = LSTM_model(actions)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

# 训练模型
epochs = 100
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:  # 迭代训练数据
        optimizer.zero_grad()  # 梯度清零
        inputs = inputs.to(torch.float32)
        labels = labels.to(torch.float32)
        outputs = model(inputs)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)  # 计算平均损失
    print('Epoch: {}, Training Loss: {:.4f}'.format(epoch+1, epoch_loss))

torch.save(model, 'model.pth')

Epoch: 1, Training Loss: 0.5926
Epoch: 2, Training Loss: 0.4788
Epoch: 3, Training Loss: 0.6364
Epoch: 4, Training Loss: 0.4847
Epoch: 5, Training Loss: 0.3843
Epoch: 6, Training Loss: 0.3326
Epoch: 7, Training Loss: 0.3437
Epoch: 8, Training Loss: 0.2472
Epoch: 9, Training Loss: 0.3464
Epoch: 10, Training Loss: 0.2245
Epoch: 11, Training Loss: 0.2400
Epoch: 12, Training Loss: 0.2844
Epoch: 13, Training Loss: 0.2108
Epoch: 14, Training Loss: 0.1606
Epoch: 15, Training Loss: 0.1728
Epoch: 16, Training Loss: 0.2425
Epoch: 17, Training Loss: 0.4360
Epoch: 18, Training Loss: 0.4567
Epoch: 19, Training Loss: 0.5581
Epoch: 20, Training Loss: 0.4583
Epoch: 21, Training Loss: 0.3154
Epoch: 22, Training Loss: 0.7955
Epoch: 23, Training Loss: 0.6727
Epoch: 24, Training Loss: 0.6695
Epoch: 25, Training Loss: 0.6499
Epoch: 26, Training Loss: 0.6437
Epoch: 27, Training Loss: 0.6667
Epoch: 28, Training Loss: 0.6377
Epoch: 29, Training Loss: 0.6588
Epoch: 30, Training Loss: 0.6085
Epoch: 31, Training

In [49]:
# 测试模型
model.eval()
for inputs, labels in test_loader:
    inputs = inputs.to(torch.float32)
    print(inputs.shape)
    res = model(inputs)
    for r in res:
        print("predict: {}".format(actions[np.argmax(r.detach().numpy())]))
    for label in labels:
        print("ground truth: {}".format(actions[np.argmax(label.numpy())]))

torch.Size([4, 30, 132])
predict: handwaving
predict: handwaving
predict: handwaving
predict: handwaving
ground truth: handwaving
ground truth: handwaving
ground truth: handwaving
ground truth: walking
torch.Size([4, 30, 132])
predict: walking
predict: walking
predict: walking
predict: walking
ground truth: walking
ground truth: walking
ground truth: walking
ground truth: walking
